In [1]:
from craftisan.tool_db.base import Session
from craftisan.tool_db.model import Tool, Category, SubCategory, Pocket
from sqlalchemy import or_, select

In [2]:
session = Session()

In [6]:
assigned_tools = None
search_query = ''

In [7]:

assigned_tool_ids = assigned_tools or (
    select(Tool.id)
    .join(Pocket, Pocket.toolId == Tool.id)
)

result = (
    session.query(Category, SubCategory, Tool)
    .join(SubCategory, Category.subcategories)
    .join(Tool, SubCategory.tools)
    .filter(
        or_(
            Tool.id.like(f'%{search_query}%'),
            Tool.description.like(f'%{search_query}%'),
        ),
        # Tool.id.notin_(subquery),
        Tool.id.notin_(assigned_tool_ids),
    )
    .all()
)

In [8]:
result

[([1] Category Бургии, [1] Category Глухо пробиване, ToolModel[3]),
 ([1] Category Бургии, [2] Category Проходно пробиване, ToolModel[4]),
 ([1] Category Бургии, [2] Category Проходно пробиване, ToolModel[5]),
 ([1] Category Бургии, [2] Category Проходно пробиване, ToolModel[7]),
 ([1] Category Бургии, [2] Category Проходно пробиване, ToolModel[8]),
 ([1] Category Бургии, [1] Category Глухо пробиване, ToolModel[12]),
 ([1] Category Бургии, [2] Category Проходно пробиване, ToolModel[13]),
 ([1] Category Бургии, [2] Category Проходно пробиване, ToolModel[14]),
 ([2] Category Фрези, [3] Category Фрезери, ToolModel[3005]),
 ([2] Category Фрези, [4] Category Профилни, ToolModel[4000]),
 ([2] Category Фрези, [4] Category Профилни, ToolModel[4002]),
 ([2] Category Фрези, [4] Category Профилни, ToolModel[4003]),
 ([2] Category Фрези, [5] Category Многопрофилни, ToolModel[5000]),
 ([2] Category Фрези, [5] Category Многопрофилни, ToolModel[5002]),
 ([3] Category Агрегати, [6] Category Наклоняеми

In [9]:
result_grouped = {}

for category, subcategory, tool in result:
    
    if category.id not in result_grouped:
        result_grouped[category.id] = {
            "id": category.id,
            "nodeType": "category",
            "displayName": category.fallbackText,
            "icon": category.icon,
            "subcategories": tuple(
                {
                    "id": subcat.id,
                    "nodeType": "subcategory",
                    "displayName": subcat.fallbackText,
                    "icon": subcat.icon,
                    "tools": tuple(
                        {
                            "id": tool.id,
                            "nodeType": "tool",
                            "displayName": str(tool),
                            "icon": tool.icon,
                        }
                        for tool in subcat.tools
                    ),
                }
                for subcat in category.subcategories
            ),
        }

    # No need to check for duplication, just append to the existing structure
    result_grouped[category.id]["subcategories"] += tuple(
        {
            "id": subcategory.id,
            "nodeType": "subcategory",
            "displayName": subcategory.fallbackText,
            "icon": subcategory.icon,
            "tools": tuple(
                {
                    "id": tool.id,
                    "nodeType": "tool",
                    "displayName": str(tool),
                    "icon": tool.icon,
                }
                for tool in subcategory.tools
            ),
        }
    )


In [11]:
import json

In [12]:
with open('test_unasigned.json', 'w+') as f:
    json.dump(result_grouped, f)

In [13]:
tools = (
    session.query(Tool)
    .filter(
        or_(
            Tool.id.like(f'%{search_query}%'),
            Tool.description.like(f'%{search_query}%'),
        ),
        Tool.id.notin_(assigned_tool_ids),
    )
    .all()
)

data = {
    tool.subcategory.categoryId: {
            "id": tool.subcategory.categoryId,
            "nodeType": "category",
            "displayName": tool.subcategory.category.fallbackText,
            "icon": tool.subcategory.category.icon,
            "subcategories": {
                "id": tool.subcategory.id,
                "nodeType": "subcategory",
                "displayName": tool.subcategory.fallbackText,
                "icon": tool.subcategory.icon,
                "tools": tuple({
                    "id": tool.id,
                    "nodeType": "tool",
                    "displayName": str(tool),
                    "icon": tool.icon,
                } for tool in tools),
            }
        } for tool in tools
    }


In [15]:
with open('test_generator.json', 'w+') as f:
    json.dump(data, f)

In [16]:
tools_result = (
    session.query(Tool)
    .filter(
        or_(
            Tool.id.like(f'%{search_query}%'),
            Tool.description.like(f'%{search_query}%'),
        ),
        Tool.id.notin_(assigned_tool_ids),
    )
    .all()
)

result_grouped = {}

for tool in tools_result:
    # Assuming each tool is associated with a single subcategory and category
    subcategory = tool.subcategory
    category = subcategory.category

    if category.id not in result_grouped:
        result_grouped[category.id] = {
            "id": category.id,
            "nodeType": "category",
            "displayName": category.fallbackText,
            "icon": category.icon,
            "subcategories": [],
        }

    subcategory_entry = {
        "id": subcategory.id,
        "nodeType": "subcategory",
        "displayName": subcategory.fallbackText,
        "icon": subcategory.icon,
        "tools": [],
    }

    # No need to check for duplication, just append to the existing structure
    subcategory_entry["tools"].append({
        "id": tool.id,
        "nodeType": "tool",
        "displayName": str(tool),
        "icon": tool.icon,
    })

    result_grouped[category.id]["subcategories"].append(subcategory_entry)


In [17]:
with open('test_chat_v2.json', 'w+') as f:
    json.dump(data, f)

In [ ]:
for tool in tools:
    print(tool.subcategory)

In [20]:
tools_by_subcategory = {}

for tool in tools_result:
    subcategory = tool.subcategory

    if subcategory.id not in tools_by_subcategory:
        tools_by_subcategory[subcategory.id] = {
            "id": subcategory.id,
            "nodeType": "subcategory",
            "displayName": subcategory.fallbackText,
            "icon": subcategory.icon,
            "tools": [],
        }

    # Add the tool to the subcategory
    tools_by_subcategory[subcategory.id]["tools"].append({
        "id": tool.id,
        "nodeType": "tool",
        "displayName": str(tool),
        "icon": tool.icon,
    })



In [31]:
tool_tree = {}

for tool in tools_result:
    subcategory = tool.subcategory
    category = subcategory.category

    if category.id not in tool_tree:
        tool_tree[category.id] = {
            "id": category.id,
            "nodeType": "category",
            "displayName": category.fallbackText,
            "icon": category.icon,
            "subcategories": [],
            "subcat": {},
        }

    if subcategory.id not in tool_tree[category.id]['subcat']:
        tool_tree[category.id]['subcat'][subcategory.id] = {
            "id": subcategory.id,
            "nodeType": "subcategory",
            "displayName": subcategory.fallbackText,
            "icon": subcategory.icon,
            "tools": [],
        }

    # Add the tool to the subcategory
    tool_tree[category.id]['subcat'][subcategory.id]["tools"].append({
        "id": tool.id,
        "nodeType": "tool",
        "displayName": str(tool),
        "icon": tool.icon,
    })

for cat in tool_tree.values():
    subcat_dict = cat.get('subcat', {})
    cat["subcategories"] = tuple(subcat_dict.values())
    del cat["subcat"]

In [29]:
for cat in tool_tree.values():
    subcat_dict = cat.get('subcat', {})
    cat["subcategories"] = tuple(subcat_dict.values())
    del cat["subcat"]    

In [32]:
with open('test_tree_id2.json', 'w+') as f:
    json.dump(tool_tree, f)

In [33]:
from craftisan.tool_db.queries import getUnassignedTools

In [34]:
tree = getUnassignedTools()

In [36]:
with open('getUnassignedTools.json', 'w') as f:
    json.dump(tree, f)